In [ ]:
!pip install geopandas

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import geopandas as gpd
rivers = gpd.read_file("Rivers_Data/Rivers.shp")
rivers.head()

In [ ]:
rivers.crs

load submission_template as test data to join the river data

In [ ]:
test_df = pd.read_csv("submission_template.csv")
test_df.head()

In [ ]:
# define longtitude and latitude column names
lon_col = "Longitude"
lat_col = "Latitude"

test_gdf = gpd.GeoDataFrame(
    test_df,
    geometry=gpd.points_from_xy(test_df[lon_col], test_df[lat_col]),
    crs="EPSG:4326"  # 經緯度
)

print(test_gdf.crs)
test_gdf.head()

In [ ]:
# Check for the validation
print(test_gdf[[lon_col, lat_col]].describe())
print(test_gdf[[lon_col, lat_col]].isna().sum())

In [ ]:
# Project to the meter coordinate system
test_m = test_gdf.to_crs("EPSG:3857")
river_m = rivers.to_crs("EPSG:3857")

In [ ]:
river_m.columns

In [ ]:
# Join the points with the nearest river
matched_df = gpd.sjoin_nearest(
    test_m,
    river_m,
    how="left",
    distance_col="dist_to_river_m"
)

matched_df.head()

In [ ]:
matched_df["dist_to_river_m"].describe(percentiles=[0.5,0.9,0.95,0.99])

In [ ]:
# Set the limit to 5km to avoid illogical join. 
max_dist = 5000  # 5km
matched_df.loc[matched_df["dist_to_river_m"] > max_dist] = None

In [ ]:
out = matched.drop(columns="geometry") 
out.to_csv("water_with_river_features.csv", index=False)


## Dams

In [ ]:
import geopandas as gpd
dams = gpd.read_file("Dams_Data/South_Africa_Dams.shp")
dams.head()